In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import h5py

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from matplotlib.pyplot import plot,savefig
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")
from read_count import read_data

import seaborn as sns

In [2]:
def plot_cluster(df, method_name, n, y_true, ax):
    
    """
        n: n-th method in [CIDR, scDeepCluster, DESC, graph-sc, scGMAAE, scVI, SCCAF, ADClust, scAce]
    """
   
    if method_name == 'Seurat':  
        y_pred = np.array(df['Cluster'])
        umap_1 = np.array(df['UMAP_1']).reshape(len(y_pred), 1)
        umap_2 = np.array(df['UMAP_2']).reshape(len(y_pred), 1)
        umap = np.concatenate((umap_1, umap_2), axis=1)
        
    elif method_name == 'CIDR':
        umap = umap_all[n]
        y_pred = np.array(df['Cluster'])
    
    elif method_name == 'scAce':
        umap = umap_all[n]
        y_pred = df['Clusters'][-1][-1]
        
    elif method_name in ['SCCAF', 'ADClust']:
        umap = umap_all[n]
        y_pred = df['Clusters'][-1]
        
    else:
        umap = umap_all[n]
        y_pred = df['Clusters']
    
    
    y_pred = np.asarray(y_pred, dtype='int').squeeze()
    
    if method_name == 'scGMAAE':
        ari = np.round(df['ARI'], 2)
        nmi = np.round(df['NMI'], 2)
    else: 
        ari = np.round(metrics.adjusted_rand_score(y_pred, y_true), 2)
        nmi = np.round(metrics.normalized_mutual_info_score(y_pred, y_true), 2)
    print('Method: {}, ARI={}, NMI={}'.format(method_name, ari, nmi))
    
        
    adata = sc.AnnData(pd.DataFrame(np.random.rand(len(y_pred), 1)))
    adata.obs['pred'] = y_pred
    adata.obs['pred'] = adata.obs['pred'].astype(str).astype('category')
    adata.obs['true'] = y_true
    adata.obs['true'] = adata.obs['true'].astype(str).astype('category')

    adata.obsm['X_umap'] = umap
    
    K = len(np.unique(y_pred))
    
    sc.pl.umap(adata, color=['pred'], ax=ax, show=False, legend_loc='None', size=8)
    ax.set_title('K={}'.format(K), fontsize=16.5, family='Arial')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [3]:
fig = plt.figure(figsize=(15, 9))
sub_figs = fig.subfigures(3, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):     
    axs.append(sub_fig.subplots(1, 4))
    
axs = np.array(axs)

# Article

## Human pancreas

In [4]:
data_mat = h5py.File('dataset/Human_p.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

In [5]:
seurat = pd.read_csv('results/default/human/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/human/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/human/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/human/DESC_wo_sample.npz')
graphsc = np.load('results/default/human/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/human/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/human/scVI_wo_sample.npz')
sccaf = np.load('results/default/human/SCCAF_wo_sample.npz')
adclust = np.load('results/default/human/ADClust_wo_sample.npz')
scace = np.load('results/default/human/scAce_wo_sample.npz')

In [6]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])
        

umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))   

In [7]:
umap_all = np.load("umap/umap_f3_human.npz")['UMAP']
# np.savez("umap/umap_f3_human.npz", UMAP=umap_all)

In [8]:
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[0][0])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[0][1])
plot_cluster(desc, 'DESC', 2, y_true, axs[0][2])
plot_cluster(scace, 'scAce', 8, y_true, axs[0][3])

Method: scDeepCluster, ARI=0.57, NMI=0.74
Method: graph-sc, ARI=0.71, NMI=0.82
Method: DESC, ARI=0.57, NMI=0.8
Method: scAce, ARI=0.9, NMI=0.88


## Human PBMC

In [9]:
data_mat = h5py.File('dataset/Human_PBMC.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

In [10]:
seurat = pd.read_csv('results/default/pbmc/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/pbmc/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/pbmc/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/pbmc/DESC_wo_sample.npz')
graphsc = np.load('results/default/pbmc/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/pbmc/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/pbmc/scVI_wo_sample.npz')
sccaf = np.load('results/default/pbmc/SCCAF_wo_sample.npz')
adclust = np.load('results/default/pbmc/ADClust_wo_sample.npz')
scace = np.load('results/default/pbmc/scAce_wo_sample.npz')

In [11]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])
        

umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))   

In [12]:
umap_all = np.load("umap/umap_f3_pbmc.npz")['UMAP']
# np.savez("umap/umap_f3_pbmc.npz", UMAP=umap_all)

In [13]:
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[1][0])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[1][1])
plot_cluster(desc, 'DESC', 2, y_true, axs[1][2])
plot_cluster(scace, 'scAce', 8, y_true, axs[1][3])

Method: scDeepCluster, ARI=0.79, NMI=0.79
Method: graph-sc, ARI=0.67, NMI=0.74
Method: DESC, ARI=0.63, NMI=0.74
Method: scAce, ARI=0.81, NMI=0.82


## Human kidney

In [14]:
data_mat = h5py.File('dataset/Human_k.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

In [15]:
seurat = pd.read_csv('results/default/kidney/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/kidney/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/kidney/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/kidney/DESC_wo_sample.npz')
graphsc = np.load('results/default/kidney/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/kidney/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/kidney/scVI_wo_sample.npz')
sccaf = np.load('results/default/kidney/SCCAF_wo_sample.npz')
adclust = np.load('results/default/kidney/ADClust_wo_sample.npz')
scace = np.load('results/default/kidney/scAce_wo_sample.npz')

In [16]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])
        

umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))   

In [17]:
umap_all = np.load("umap/umap_f3_kidney.npz")['UMAP']
# np.savez("umap/umap_f3_kidney.npz", UMAP=umap_all)

In [18]:
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[2][0])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[2][1])
plot_cluster(desc, 'DESC', 2, y_true, axs[2][2])
plot_cluster(scace, 'scAce', 8, y_true, axs[2][3])

Method: scDeepCluster, ARI=0.48, NMI=0.66
Method: graph-sc, ARI=0.68, NMI=0.79
Method: DESC, ARI=0.55, NMI=0.76
Method: scAce, ARI=0.65, NMI=0.76


In [19]:
fig

<Figure size 1500x900 with 12 Axes>

In [20]:
plt.savefig('Figures0418/Figure3B.svg', dpi=300, format='svg', bbox_inches='tight')

# Supplementary

In [21]:
fig = plt.figure(figsize=(24, 10))
sub_figs = fig.subfigures(3, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):     
    axs.append(sub_fig.subplots(1, 6))
    
axs = np.array(axs)

## Human pancreas

In [22]:
data_mat = h5py.File('dataset/Human_p.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

seurat = pd.read_csv('results/default/human/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/human/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/human/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/human/DESC_wo_sample.npz')
graphsc = np.load('results/default/human/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/human/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/human/scVI_wo_sample.npz')
sccaf = np.load('results/default/human/SCCAF_wo_sample.npz')
adclust = np.load('results/default/human/ADClust_wo_sample.npz')
scace = np.load('results/default/human/scAce_wo_sample.npz')

y_true_scgmaae = scgmaae['Labels']
umap_all = np.load("umap/umap_f3_human.npz")['UMAP']

In [23]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[0][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[0][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[0][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[0][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[0][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[0][5])

Method: CIDR, ARI=0.56, NMI=0.65
Method: SCCAF, ARI=0.54, NMI=0.77
Method: scGMAAE, ARI=0.47, NMI=0.58
Method: scVI, ARI=0.7, NMI=0.83
Method: Seurat, ARI=0.57, NMI=0.79
Method: ADClust, ARI=0.79, NMI=0.76


## Human PBMC

In [24]:
data_mat = h5py.File('dataset/Human_PBMC.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

seurat = pd.read_csv('results/default/pbmc/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/pbmc/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/pbmc/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/pbmc/DESC_wo_sample.npz')
graphsc = np.load('results/default/pbmc/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/pbmc/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/pbmc/scVI_wo_sample.npz')
sccaf = np.load('results/default/pbmc/SCCAF_wo_sample.npz')
adclust = np.load('results/default/pbmc/ADClust_wo_sample.npz')
scace = np.load('results/default/pbmc/scAce_wo_sample.npz')

y_true_scgmaae = scgmaae['Labels']
umap_all = np.load("umap/umap_f3_pbmc.npz")['UMAP']

In [25]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[1][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[1][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[1][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[1][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[1][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[1][5])

Method: CIDR, ARI=0.64, NMI=0.72
Method: SCCAF, ARI=0.64, NMI=0.73
Method: scGMAAE, ARI=0.6, NMI=0.67
Method: scVI, ARI=0.52, NMI=0.67
Method: Seurat, ARI=0.59, NMI=0.72
Method: ADClust, ARI=0.41, NMI=0.63


## Human kidney

In [26]:
data_mat = h5py.File('dataset/Human_k.h5')
y_true = np.array(data_mat['Y'], dtype='int')
data_mat.close()

seurat = pd.read_csv('results/default/kidney/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/kidney/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/kidney/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/kidney/DESC_wo_sample.npz')
graphsc = np.load('results/default/kidney/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/kidney/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/kidney/scVI_wo_sample.npz')
sccaf = np.load('results/default/kidney/SCCAF_wo_sample.npz')
adclust = np.load('results/default/kidney/ADClust_wo_sample.npz')
scace = np.load('results/default/kidney/scAce_wo_sample.npz')

y_true_scgmaae = scgmaae['Labels']
umap_all = np.load("umap/umap_f3_kidney.npz")['UMAP']

In [27]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[2][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[2][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[2][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[2][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[2][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[2][5])

Method: CIDR, ARI=0.26, NMI=0.45
Method: SCCAF, ARI=0.37, NMI=0.7
Method: scGMAAE, ARI=0.57, NMI=0.65
Method: scVI, ARI=0.51, NMI=0.72
Method: Seurat, ARI=0.4, NMI=0.66
Method: ADClust, ARI=0.46, NMI=0.64


In [28]:
fig

<Figure size 2400x1000 with 18 Axes>

In [29]:
plt.savefig('Figures0418/FigureS1A.png', dpi=800, format='png', bbox_inches='tight')

In [30]:
fig = plt.figure(figsize=(25, 8))
sub_figs = fig.subfigures(4, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):     
    axs.append(sub_fig.subplots(1, 10))
    
axs = np.array(axs)

## Mouse ES

In [31]:
mat, obs, var, uns = read_data('dataset/Mouse_E.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true = np.unique(cell_name, return_inverse=True)

In [32]:
seurat = pd.read_csv('results/default/Klein/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/Klein/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/Klein/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/Klein/DESC_wo_sample.npz')
graphsc = np.load('results/default/Klein/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/Klein/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/Klein/scVI_wo_sample.npz')
sccaf = np.load('results/default/Klein/SCCAF_wo_sample.npz')
adclust = np.load('results/default/Klein/ADClust_wo_sample.npz')
scace = np.load('results/default/Klein/scAce_wo_sample.npz')

In [33]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])
        

umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))   

In [34]:
umap_all = np.load("umap/umap_f3_klein.npz")['UMAP']
# np.savez("umap/umap_f3_klein.npz", UMAP=umap_all)

In [35]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[0][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[0][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[0][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[0][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[0][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[0][5])
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[0][6])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[0][7])
plot_cluster(desc, 'DESC', 2, y_true, axs[0][8])
plot_cluster(scace, 'scAce', 8, y_true, axs[0][9])

Method: CIDR, ARI=0.68, NMI=0.66
Method: SCCAF, ARI=0.82, NMI=0.86
Method: scGMAAE, ARI=0.78, NMI=0.73
Method: scVI, ARI=0.62, NMI=0.74
Method: Seurat, ARI=0.52, NMI=0.73
Method: ADClust, ARI=0.73, NMI=0.69
Method: scDeepCluster, ARI=0.49, NMI=0.63
Method: graph-sc, ARI=0.78, NMI=0.76
Method: DESC, ARI=0.9, NMI=0.9
Method: scAce, ARI=0.9, NMI=0.9


## Mouse hypothalamus

In [36]:
mat, obs, var, uns = read_data('dataset/Mouse_h.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_1 = np.unique(cell_name, return_inverse=True)

In [37]:
adata = sc.AnnData(x)
adata.obs['celltype'] = y_true_1
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)

y_true_2 = np.array(adata.obs['celltype']).squeeze()

In [38]:
seurat = pd.read_csv('results/default/chen/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/chen/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/chen/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/chen/DESC_wo_sample.npz')
graphsc = np.load('results/default/chen/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/chen/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/chen/scVI_wo_sample.npz')
sccaf = np.load('results/default/chen/SCCAF_wo_sample.npz')
adclust = np.load('results/default/chen/ADClust_wo_sample.npz')
scace = np.load('results/default/chen/scAce_wo_sample.npz')

In [39]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])
        

umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))   

In [40]:
umap_all = np.load("umap/umap_f3_chen.npz")['UMAP']
# np.savez("umap/umap_f3_chen.npz", UMAP=umap_all)

In [41]:
plot_cluster(cidr, 'CIDR', 0, y_true_1, axs[1][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true_2, axs[1][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[1][2])
plot_cluster(scvi, 'scVI', 5, y_true_1, axs[1][3])
plot_cluster(seurat, 'Seurat', None, y_true_2, axs[1][4])
plot_cluster(adclust, 'ADClust', 7, y_true_1, axs[1][5])
plot_cluster(scd, 'scDeepCluster', 1, y_true_1, axs[1][6])
plot_cluster(graphsc, 'graph-sc', 3, y_true_1, axs[1][7])
plot_cluster(desc, 'DESC', 2, y_true_2, axs[1][8])
plot_cluster(scace, 'scAce', 8, y_true_2, axs[1][9])

Method: CIDR, ARI=0.36, NMI=0.42
Method: SCCAF, ARI=0.6, NMI=0.77
Method: scGMAAE, ARI=0.49, NMI=0.69
Method: scVI, ARI=0.57, NMI=0.75
Method: Seurat, ARI=0.88, NMI=0.8
Method: ADClust, ARI=0.78, NMI=0.78
Method: scDeepCluster, ARI=0.61, NMI=0.74
Method: graph-sc, ARI=0.4, NMI=0.73
Method: DESC, ARI=0.66, NMI=0.8
Method: scAce, ARI=0.84, NMI=0.79


## Mouse kidney

In [42]:
mat, obs, var, uns = read_data('dataset/Mouse_k.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true = np.unique(cell_name, return_inverse=True)

In [43]:
seurat = pd.read_csv('results/default/Adam/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/Adam/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/Adam/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/Adam/DESC_wo_sample.npz')
graphsc = np.load('results/default/Adam/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/Adam/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/Adam/scVI_wo_sample.npz')
sccaf = np.load('results/default/Adam/SCCAF_wo_sample.npz')
adclust = np.load('results/default/Adam/ADClust_wo_sample.npz')
scace = np.load('results/default/Adam/scAce_wo_sample.npz')

In [44]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])

In [45]:
umap_all = np.load("umap/umap_f3_adam.npz")['UMAP']
# np.savez("umap/umap_f3_adam.npz", UMAP=umap_all)

In [46]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[2][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[2][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[2][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[2][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[2][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[2][5])
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[2][6])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[2][7])
plot_cluster(desc, 'DESC', 2, y_true, axs[2][8])
plot_cluster(scace, 'scAce', 8, y_true, axs[2][9])

Method: CIDR, ARI=0.05, NMI=0.12
Method: SCCAF, ARI=0.56, NMI=0.74
Method: scGMAAE, ARI=0.68, NMI=0.75
Method: scVI, ARI=0.67, NMI=0.8
Method: Seurat, ARI=0.64, NMI=0.78
Method: ADClust, ARI=0.47, NMI=0.6
Method: scDeepCluster, ARI=0.63, NMI=0.73
Method: graph-sc, ARI=0.6, NMI=0.71
Method: DESC, ARI=0.64, NMI=0.8
Method: scAce, ARI=0.93, NMI=0.91


## Turtle brain

In [47]:
mat, obs, var, uns = read_data('dataset/Turtle_b.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true = np.unique(cell_name, return_inverse=True)

In [48]:
seurat = pd.read_csv('results/default/turtle/Seurat_wo_sample.csv', header=0, index_col=0)
cidr = pd.read_csv('results/default/turtle/CIDR_wo_sample.csv', header=0, index_col=0)
scd = np.load('results/default/turtle/scDeepCluster_wo_sample.npz')
desc = np.load('results/default/turtle/DESC_wo_sample.npz')
graphsc = np.load('results/default/turtle/graphsc_wo_sample.npz')
scgmaae = np.load('results/default/turtle/scGMAAE_wo_sample.npz')
scvi = np.load('results/default/turtle/scVI_wo_sample.npz')
sccaf = np.load('results/default/turtle/SCCAF_wo_sample.npz')
adclust = np.load('results/default/turtle/ADClust_wo_sample.npz')
scace = np.load('results/default/turtle/scAce_wo_sample.npz')

In [49]:
y_true_scgmaae = scgmaae['Labels']

In [ ]:
methods = [cidr, scd, desc, graphsc, scgmaae, scvi, sccaf, adclust, scace]
embedding = []
for i, method in enumerate(methods):
    
    if i == 0:
        embedding.append(method[['Axis.1', 'Axis.2', 'Axis.3', 'Axis.4']])
        
    elif i == 1:
        embedding.append(method['Embedded'])
    
    elif i == len(methods) - 2 or i == len(methods) - 1:
        embedding.append(method['Embedding'][-1])
    else:
        embedding.append(method['Embedding'])

In [50]:
umap_all = np.load("umap/umap_f3_turtle.npz")['UMAP']
# np.savez("umap/umap_f3_turtle.npz", UMAP=umap_all)

In [51]:
plot_cluster(cidr, 'CIDR', 0, y_true, axs[3][0])
plot_cluster(sccaf, 'SCCAF', 6, y_true, axs[3][1])
plot_cluster(scgmaae, 'scGMAAE', 4, y_true_scgmaae, axs[3][2])
plot_cluster(scvi, 'scVI', 5, y_true, axs[3][3])
plot_cluster(seurat, 'Seurat', None, y_true, axs[3][4])
plot_cluster(adclust, 'ADClust', 7, y_true, axs[3][5])
plot_cluster(scd, 'scDeepCluster', 1, y_true, axs[3][6])
plot_cluster(graphsc, 'graph-sc', 3, y_true, axs[3][7])
plot_cluster(desc, 'DESC', 2, y_true, axs[3][8])
plot_cluster(scace, 'scAce', 8, y_true, axs[3][9])

Method: CIDR, ARI=0.48, NMI=0.63
Method: SCCAF, ARI=0.39, NMI=0.74
Method: scGMAAE, ARI=0.48, NMI=0.68
Method: scVI, ARI=0.5, NMI=0.75
Method: Seurat, ARI=0.56, NMI=0.74
Method: ADClust, ARI=0.62, NMI=0.73
Method: scDeepCluster, ARI=0.7, NMI=0.76
Method: graph-sc, ARI=0.51, NMI=0.72
Method: DESC, ARI=0.68, NMI=0.84
Method: scAce, ARI=0.71, NMI=0.71


In [52]:
fig

<Figure size 2500x800 with 40 Axes>

In [53]:
plt.savefig('Figures0418/FigureS2A.png', dpi=800, format='png', bbox_inches='tight')